In [1]:
!pip install neo4j==5.24.0

  Using cached neo4j-5.24.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
Using cached neo4j-5.24.0-py3-none-any.whl (294 kB)
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)


In [2]:
from neo4j import GraphDatabase

In [3]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    # Queries
    def create_node(self, label, properties):
        with self.driver.session() as session:
            session.run(f"CREATE (n:{label} $properties", properties=properties)


    def degree_centrality(self):
        """
        Calculates the degree centrality for nodes in the graph, returning nodes with their centrality scores.

        Returns:
            list: A list of tuples containing (AgentId, score) for each node.

        Usage:
            Use this method to analyze the structure of the network and find key nodes
            that might be influential in information propagation or decision-making.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.degree.stream('myGraph')
            YIELD nodeId, score
            WHERE gds.util.asNode(nodeId).AgentId IS NOT NULL
            RETURN gds.util.asNode(nodeId).AgentId AS AgentId, score
            ORDER BY score DESC
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"AgentId": record["AgentId"], "score": record["score"]} for record in result]


    def detect_client_communities(self):
        """
        Detects communities of clients in the graph using the label propagation algorithm,
        returning community identifiers and their members.

        Returns:
            list: A list of tuples containing (communityId, Members).

        Usage:
            Use this method to understand the structure of the client base or in social network
            analysis where it is useful to identify groups of users with common interests.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.labelPropagation.stream('myGraph')
            YIELD nodeId, communityId
            RETURN communityId, collect(gds.util.asNode(nodeId).ClientId) AS Members
            ORDER BY size(Members) DESC
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"communityId": record["communityId"], "Members": record["Members"]} for record in result]


    def detect_agent_communities(self):
        """
        Detects communities of agents in the graph using the Louvain algorithm,
        returning community identifiers and their members.

        Returns:
            list: A list of tuples containing (communityId, Members).

        Usage:
            Use this method when you need to perform a deep analysis of the relationships
            among agents, especially in contexts where collaboration or interaction
            among them is relevant.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.louvain.stream('myGraph')
            YIELD nodeId, communityId
            RETURN communityId, collect(gds.util.asNode(nodeId).AgentId) AS Members
            ORDER BY size(Members) DESC
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"communityId": record["communityId"], "Members": record["Members"]} for record in result]


    def node_similarity(self):
        """
        Calculates the similarity between nodes in the graph, returning pairs of nodes with their similarity scores.

        Returns:
            list: A list of tuples containing (Node1, Node2, similarity) for each pair.

        Usage:
            Use this method in recommendation systems or network analysis when you want
            to identify nodes that have similar characteristics or behaviors.
        """
        with self.driver.session() as session:
            result = session.run(f"""
            CALL gds.nodeSimilarity.stream('myGraph')
            YIELD node1, node2, similarity
            WHERE gds.util.asNode(node1).ClientId IS NOT NULL
            AND gds.util.asNode(node2).ClientId IS NOT NULL
            RETURN gds.util.asNode(node1).ClientId AS Node1, gds.util.asNode(node2).ClientId AS Node2, similarity
            ORDER BY similarity DESC
            LIMIT 10
            """)

            # Convertir el resultado a una lista de diccionarios
            return [{"Node1": record["Node1"], "Node2": record["Node2"], "similarity": record["similarity"]} for record in result]


    def get_nodes(self, label):
        with self.driver.session() as session:
            result = session.run(f"MATCH (n:{label}) RETURN n")
            return [record["n"] for record in result]


In [4]:
uri = "bolt://localhost:7687"  # Cambia esto si tu URI es diferente
user = "neo4j"                   # Nombre de usuario
password = "llmHackaton2024"       # Contraseña

# Uso de la clase
neo4j_conn = Neo4jConnection(uri, user, password)

In [5]:
degree_centrality  = neo4j_conn.degree_centrality()
similarity_results = neo4j_conn.node_similarity()
community_results = neo4j_conn.detect_client_communities()
agent_community_results = neo4j_conn.detect_agent_communities()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n            CALL gds.labelPropagation.stream('myGraph')\n            YIELD nodeId, communityId\n            RETURN communityId, collect(gds.util.asNode(nodeId).ClientId) AS Members\n            ORDER BY size(Members) DESC\n            "
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.AggregationSkippedNull} {category: UNRECOGNIZED} {title: The query contains an aggregation function that skips null values.} {description: null value eliminated in set function.} {position: None} for query: "\n            CALL gds.louvain.stream('myGraph')\n            YIELD nodeId, communityId\n            RETURN communityId, collect(gds.util.asNode(

In [6]:
degree_centrality

[{'AgentId': '3', 'score': 11.0},
 {'AgentId': '2', 'score': 10.0},
 {'AgentId': '7', 'score': 8.0},
 {'AgentId': '11', 'score': 8.0},
 {'AgentId': '1', 'score': 7.0},
 {'AgentId': '9', 'score': 7.0},
 {'AgentId': '10', 'score': 7.0},
 {'AgentId': '6', 'score': 6.0},
 {'AgentId': '12', 'score': 5.0},
 {'AgentId': '19', 'score': 5.0},
 {'AgentId': '5', 'score': 4.0},
 {'AgentId': '18', 'score': 4.0},
 {'AgentId': '4', 'score': 3.0},
 {'AgentId': '8', 'score': 3.0},
 {'AgentId': '14', 'score': 3.0},
 {'AgentId': '13', 'score': 2.0},
 {'AgentId': '15', 'score': 2.0},
 {'AgentId': '16', 'score': 2.0},
 {'AgentId': '20', 'score': 2.0},
 {'AgentId': '17', 'score': 1.0},
 {'AgentId': '1', 'score': 0.0},
 {'AgentId': '1', 'score': 0.0},
 {'AgentId': '3', 'score': 0.0},
 {'AgentId': '4', 'score': 0.0},
 {'AgentId': '3', 'score': 0.0},
 {'AgentId': '1', 'score': 0.0},
 {'AgentId': '2', 'score': 0.0},
 {'AgentId': '2', 'score': 0.0},
 {'AgentId': '9', 'score': 0.0},
 {'AgentId': '1', 'score': 0.0

In [7]:
similarity_results

[]

In [8]:
community_results

[{'communityId': 40, 'Members': ['1', '1']},
 {'communityId': 46, 'Members': ['7', '7']},
 {'communityId': 42, 'Members': ['3', '3']},
 {'communityId': 43, 'Members': ['8', '8']},
 {'communityId': 52, 'Members': ['13', '13']},
 {'communityId': 51, 'Members': ['12', '12']},
 {'communityId': 58, 'Members': ['19', '19']},
 {'communityId': 55, 'Members': ['16', '16']},
 {'communityId': 57, 'Members': ['18', '18']},
 {'communityId': 59, 'Members': ['20', '20']},
 {'communityId': 41, 'Members': ['2', '2']},
 {'communityId': 69, 'Members': ['4', '4']},
 {'communityId': 44, 'Members': ['5', '5']},
 {'communityId': 71, 'Members': ['6', '6']},
 {'communityId': 62, 'Members': ['9', '9']},
 {'communityId': 49, 'Members': ['10', '10']},
 {'communityId': 76, 'Members': ['11', '11']},
 {'communityId': 53, 'Members': ['14', '14']},
 {'communityId': 54, 'Members': ['15', '15']},
 {'communityId': 82, 'Members': ['17', '17']},
 {'communityId': 90, 'Members': ['25']},
 {'communityId': 68, 'Members': ['3']

In [9]:
agent_community_results

[{'communityId': 82,
  'Members': ['6',
   '7',
   '10',
   '19',
   '20',
   '7',
   '7',
   '20',
   '7',
   '7',
   '6',
   '2',
   '7',
   '10',
   '7',
   '11',
   '6',
   '4',
   '19',
   '6',
   '10',
   '19',
   '10',
   '6',
   '19',
   '10',
   '10',
   '20',
   '6',
   '19',
   '10',
   '6',
   '11']},
 {'communityId': 135,
  'Members': ['4',
   '5',
   '8',
   '14',
   '16',
   '17',
   '4',
   '1',
   '2',
   '14',
   '7',
   '16',
   '8',
   '3',
   '12',
   '16',
   '5',
   '4',
   '7',
   '5',
   '8',
   '5',
   '19',
   '5',
   '14',
   '17',
   '2']},
 {'communityId': 133,
  'Members': ['3',
   '9',
   '18',
   '3',
   '9',
   '18',
   '9',
   '3',
   '3',
   '18',
   '3',
   '3',
   '9',
   '3',
   '3',
   '9',
   '18',
   '18',
   '9',
   '3']},
 {'communityId': 136,
  'Members': ['11',
   '13',
   '1',
   '3',
   '11',
   '11',
   '3',
   '11',
   '13',
   '8',
   '11',
   '11',
   '13',
   '11',
   '2',
   '9']},
 {'communityId': 137,
  'Members': ['1',
   '15',
 

In [10]:
agent_community_results

[{'communityId': 82,
  'Members': ['6',
   '7',
   '10',
   '19',
   '20',
   '7',
   '7',
   '20',
   '7',
   '7',
   '6',
   '2',
   '7',
   '10',
   '7',
   '11',
   '6',
   '4',
   '19',
   '6',
   '10',
   '19',
   '10',
   '6',
   '19',
   '10',
   '10',
   '20',
   '6',
   '19',
   '10',
   '6',
   '11']},
 {'communityId': 135,
  'Members': ['4',
   '5',
   '8',
   '14',
   '16',
   '17',
   '4',
   '1',
   '2',
   '14',
   '7',
   '16',
   '8',
   '3',
   '12',
   '16',
   '5',
   '4',
   '7',
   '5',
   '8',
   '5',
   '19',
   '5',
   '14',
   '17',
   '2']},
 {'communityId': 133,
  'Members': ['3',
   '9',
   '18',
   '3',
   '9',
   '18',
   '9',
   '3',
   '3',
   '18',
   '3',
   '3',
   '9',
   '3',
   '3',
   '9',
   '18',
   '18',
   '9',
   '3']},
 {'communityId': 136,
  'Members': ['11',
   '13',
   '1',
   '3',
   '11',
   '11',
   '3',
   '11',
   '13',
   '8',
   '11',
   '11',
   '13',
   '11',
   '2',
   '9']},
 {'communityId': 137,
  'Members': ['1',
   '15',
 